In [1]:
import pandas as pd
import pyspark
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
visitas = pd.read_csv('Sources/fiuba_3_vistas_concat.csv').drop('Unnamed: 0', axis=1).drop('timestamp',axis=1)
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv').drop('Unnamed: 0', axis=1).drop('fechapostulacion', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)

In [4]:
visitas.rename(columns={'idAviso':'idaviso'},inplace=True)
visitas['peso'] = 1
postulaciones['peso'] = 2

In [5]:
concat = pd.concat([postulaciones,visitas])

In [6]:
concat.head()

,idaviso,idpostulante,peso
0,1112248724,NjlD,2
1,1112286523,ZaO5,2
2,1112272060,ZaO5,2
3,1112288401,ZaO5,2
4,1112300563,ZaO5,2


In [7]:
t = concat.drop_duplicates(['idaviso','idpostulante'], keep=False)

In [8]:
t.loc[t['peso']==2].shape

(3347693, 3)

In [9]:
t.loc[t['peso']==1].shape

(3261695, 3)

In [10]:
concat.loc[concat['peso']==2].shape

(8311264, 3)

In [11]:
concat.loc[concat['peso']==1].shape

(18368813, 3)

In [12]:
concat = concat.drop_duplicates(['idaviso','idpostulante'])

In [13]:
pesos = concat['peso']
concat.drop('peso', axis=1, inplace=True)

In [14]:
concat = concat.merge(avisos, on='idaviso')
concat = concat.merge(postulantes, on='idpostulante')

In [15]:
concat.drop(['idaviso','idpostulante','titulo','descripcion'], axis=1, inplace=True)
concat.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel
0,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,188,37.0,FEM,12
1,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,188,27.0,MASC,11
2,Full-time,Junior,Comunicacion,RH Talentum,76,27.0,MASC,11
3,Full-time,Senior / Semi-Senior,Comunicacion,Jones Lang LaSalle,76,27.0,MASC,11
4,Full-time,Senior / Semi-Senior,Comunicacion,Assistem,76,27.0,MASC,11


In [ ]:
le = preprocessing.LabelEncoder()
for col in concat.columns:
    if not concat[col].dtype == 'O': continue
    concat[col] = concat[col].fillna('na')
    le.fit(concat[col])
    concat[col] = le.transform(concat[col])
concat.fillna('-1')

In [ ]:
kmeanModel = KMeans(n_clusters=228, init='k-means++', max_iter=100, n_init=30, n_jobs=-1)
kmeanModel = kmeanModel.fit(concat)
clusters = kmeanModel.predict(df)
concat['cluster_post'] = clusters

In [16]:
concat.to_csv('Sources/test_kmeans_post.csv')